# Alchemical free energy setup


<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

Authors:   
[Antonia Mey -- @ppxasjsm](https://github.com/ppxasjsm)   
[Lester Hedges -- @lohedges](https://github.com/lohedges)

## Learning objectives:
- Setup an alchemical solvation free energy simulation using BioSimSpace and SOMD
- Setup an alchemical solvation free energy simulation using BioSimSpace and Gromacs
- Setup an alchemical binding free energy simulation for Gromacs and SOMD using BioSimSpace

You will be using the following functions in BioSimSpace:

- `BSS.IO.readMolecules()` To load the molecules
- `BSS.Parameters.gaff` To parameterise molecules using the Generalised Amber Force Field (GAFF)
- `BSS.Parameters.ff14SB` To parameterise a protein using FF14SB
- `BSS.Align.matchAtoms()` Maximum Common Substructure matches atoms for the morphing
- `BSS.Align.rmsdAlign()` Aligns the molecules to be morphed
- `BSS.Align.merge()` Creates a merged molecule used for alchemical simulations
- `BSS.Solvent.tip3p()` Solvates a molecule in a tip3 water box
- `BSS.Protocol.FreeEnergy()` Defines the free energy protocol
- `BSS.FreeEnergy.Solvation()` Sets up a solvation free energy process
- `BSS.FreeEnergy.Binding()` Sets up a binding free energy process


**Reading time**:
~ 30 mins

**Jupyter cheat sheet**:
- To run the currently highlighted cell and move focus to the next cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- To run the currently highlighted cell and keep focus in the same cell, hold <kbd>&#x21E7; Ctri</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- To get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;
- You can find the full documentation at [biosimspace.org](https://biosimspace.org).

## Table of Contents
1. [Working with ligands](#lig)    
   1.1 [Loading ligands](#load)   
   1.2 [Parametrising ligands](#param)   
2. [Morphing ligands](#merge)   
   2.1 [MCS](#mcs)   
   2.2 [morphed ligand](#morph)   
3. [Running solvation free energy simulation](#solv)
4. [Binding free energy simulation](#bind)   
5. [Exercises](#exerc2)   

First, let's import BioSimspace!

In [ ]:
import BioSimSpace as BSS

## 1. Free energy of solvation of ethane and methanol
<a id="lig"></a>

We want to compute the relative free energy of hydration between ethane and methanol. That is the free energy difference between hydrating an ethane molecule in water and a methanol molecule in water. (For reference, refer to the [slides](slides/CCPBioSim-FEP-tutorial.pdf)).

Below you can see a thermodynamic cycle for the relative hydration free energy of ethane and methanol:
![therm_cycle](images/Therm_cycle.png)

### 1.1 Loading ligands
<a id="load"></a>
Next, we read in the two molecules we want to perturb. In this case, this is ethane changing to methanol. You can use the BioSimSpace function `BSS.IO.readMolecules()` for this task.

In [ ]:
# We assume the molecules to perturb are the first molecules in each system. (Each file contains a single molecule.)
ethane = BSS.IO.readMolecules("input/ethane.pdb")[0]
methanol = BSS.IO.readMolecules("input/methanol.pdb")[0]

It might be nice to quickly check we are reading in the right molecules so we can visualise them using the `View` class from `BSS.Notebook`.

In [ ]:
BSS.Notebook.View("input/ethane.pdb").system()

In [ ]:
BSS.Notebook.View("input/methanol.pdb").system()

### 1.2 Parametrising molecules
<a id="param"></a>

Current we only have the coordinates information saved for the loaded molecules so the first thing we need to do is generated some forcefield parameters. In this case, we will use the Generalise Amber Force Field [(GAFF)](http://ambermd.org/antechamber/gaff.html).

In [ ]:
ethane = BSS.Parameters.gaff(ethane).getMolecule()
methanol = BSS.Parameters.gaff(methanol).getMolecule()

## 2. Creating merged system
<a id="merge"></a>
Now ethane and methanol have all the required properties for running an MD simulation of them individually. But here we are interested in creating a morphed system, or `single topology` for running an alchemical free energy calculation. In this case, two of the ethane hydrogens will turn into dummy atoms and the second carbon and the 3rd hydrogen will turn into the `OH` group of the methanol. 

### 2.1 Maximum Common Substructure (MCS)
<a id="MCS"></a>
In order to automatically figure out which atoms are common between ethane and methanol we can use the `matchAtoms()` function of BioSimSpace. This will compute a Maxmimum Common Substructure (MCS) match and return a dictionary that maps the indices of atoms in the ethane molecule to the indices of the atoms in the ethanol to which they match. An example of what and MCS match might look like is shown here:
![MCS](images/MCS.png)


In [ ]:
mapping = BSS.Align.matchAtoms(ethane, methanol)

# Mapping is a dictionary mapping atom indices in ethane to those in methanol.
print(mapping)

Once we have the mapping we can align the molecules to each other using a root mean squared displacemnt (RMSD) metric and from the alignment we can then create a merged molecule which contains all of the `single topology` information needed for the alchemical perturbation.

In [ ]:
# Align ethane to methanol based on the mapping.
ethane = BSS.Align.rmsdAlign(ethane, methanol, mapping)

# Merge the ethane and methanol based on the mapping.
merged = BSS.Align.merge(ethane, methanol, mapping)

### 2.2 Creating a morph
<a id="morph"></a>

Different software tools have different ways of running alchemical free energy calculations. If you were to use `SOMD` for the underlying free energy calculations you will automatically generate something called a `pert` file. This file contains information for how all parameters, e.g. the charges, change with between the two end states of the perturbation, $\lambda=0$ and $\lambda=1$.

Let's have a closer look at this merged molecule:

First we'll use an internal functional of BioSimSpace to extract the properties for the end state of the perturbation, $\lambda=1$. (This isn't something the user nornally needs to do, hence the function is private, i.e. _hidden_.)

In [ ]:
# Extract the lambda = 1 state and save to file.
BSS.IO.saveMolecules("test", merged._toRegularMolecule(is_lambda1=True), "pdb")

In [ ]:
# View the lambda = 1 state.
BSS.Notebook.View("test.pdb").system()

In [ ]:
!cat 'test.pdb'

In [ ]:
# Generate the SOMD perurbation (pert) file. Again, this is normally done automatically by
# BioSimSpace during the simulation set up, so the functionality is hidden by default. The
# function also returns the lambda = 0 state of the perturbation, which is used to generate
# the required coordinate and topology input files for SOMD. Here we ignore this return argument.
merged._toPertFile("ethane_methanol.pert")

In [ ]:
!head -n 20 ethane_methanol.pert

### 2.3 Solvation
<a id="solv"></a>

Before we can run a free energy simulation we will have to solvate the system. In this case, rather than passing ethane and methanol separately we will solvate the whole merged system. Here we use a cubic box with a base length of 40 Angstroms.

In [ ]:
solvated = BSS.Solvent.tip3p(molecule=merged, box=3*[40*BSS.Units.Length.angstrom])

## 3. Solvation free energy
<a id="free"></a>
We now need to define a protocol to describe the parameters used for the free energy perturbation. A simple protocol consists of a 2 fs timestep a runtime of 4 ns and using equally spaced 9 $\lambda$ windows:

In [ ]:
protocol = BSS.Protocol.FreeEnergy(timestep=2*BSS.Units.Time.femtosecond, runtime=4*BSS.Units.Time.nanosecond, num_lam=9)

Next we want to create a free energy simulation _process_ to apply our protocol to the solvated system containing the merged molecule:

In [ ]:
freenrg = BSS.FreeEnergy.Solvation(solvated, protocol, work_dir="ethane_methanol_somd")

Next you would run the alchemical free energy simulation in the following way:

`freenrg.run()`   

This only makes sense on a workstation with GPUs or GPU cloud resources or a GPU cluster. Otherwise you will have to wait for too long to run these simulations on the notebook server.

Let's have a look the `ethane_methanol` directory. In this directory you have now all the files setup and ready for simulation using SOMD as the simulation engine, which is the default.

In [ ]:
! ls ethane_methanol_somd

Notice how the `work_dir` contains two directories, `free` and `vacuum`. The `free` directory contains the files needed to simulate the perturbation for the _solvated_ system, whereas _vacuum_, unsurprisingly, contains files for the vacuum perturbation. Have a look at the content of these directories, which contain sub-directories for each lambda window of the perturbations.

In [ ]:
! ls ethane_methanol_somd/free

### 3.1. Exercises
<a id="exerc1"></a>

The exercises are announced by the keyword **Exercise** and followed by an incomplete cell.
Missing parts are indicated by:

```python
#FIXME
```

### 3.1.1. Exercise on selecting lambda windows

Above we defined a protocol with 9 $\lambda$ windows. For this system this isn't an optimal protocol and we would like to instead use 12 lambda window. Can you write down a protocol that would allow you to run 12 rather than 9 lambda windows?

In [ ]:
protocol = BSS.Protocol.FreeEnergy(timestep=2*BSS.Units.Time.femtosecond, runtime=4*BSS.Units.Time.nanosecond, num_lam=12)

### 3.1.2. Exercise on merged molecules
Previously we have set up an ethane to methanol alchemical free energy simulation. One way of assessing how good an estimate of a free energy difference from an alchemical simulation is by running the simulation in the opposite direction, i.e. methanol to ethane. Can you set up a new merged molecule and run the necessary steps for the free energy setup?

In [ ]:
mapping = BSS.Align.matchAtoms(methanol, ethane)

# Align methanol to ethane based on the mapping.
methanol = BSS.Align.rmsdAlign(methanol, ethane, mapping)

# Merge the two molecules based on the mapping.
merged_methanol = BSS.Align.merge(methanol, ethane, mapping)
solvated_methanol = BSS.Solvent.tip3p(molecule=merged_methanol, box=3*[40*BSS.Units.Length.angstrom])
freenrg_methanol = BSS.FreeEnergy.Solvation(solvated_methanol, protocol, work_dir="methanol_ethane")

### 3.1.3. Exercise on using different simulation engines

Currently, alchemical free energy simulations with SOMD and Gromacs are supported. Can you figure out how to setup and run the simulations using GROMACS rather than the default of SOMD?

**Hint**: look at the `engine` keyword of `FreeEnergy.Solvation()`. You might also want to change the working directory.

In [ ]:
freenrg_gromacs = BSS.FreeEnergy.Solvation(solvated_methanol, protocol, work_dir="methanol_ethane_gromacs", engine="gromacs")

You will notice that again in your `work_dir`, two directories were created `free` and `vacuum`. Have a look at the content of these directories. You will notice that the input files that were generated are now indeed for GROMACS and not SOMD. Take a moment to look at the config files etc, e.g.:

In [ ]:
!ls ethane_methanol_gromacs/free/lambda_0.0000

## 4. Free energy of binding
<a id="bind"></a>

So far we have done a setup for free energies of hydration. Next we'll learn how to use BioSimSpace to set up alchemical free energy simulations that can be used to compute free energies of binding. The thermodynamic cycle for the free energy of binding looks like this:

![reltherm](images/thermo_cycle_rel_eq.png)

In our case the host is Lysozyme, an antimicrobial protein, which has been studied extensivley using alchemical free energy calculations in the past.  

### 4.1 Loading the protein and ligands
Loading the protein is done in the same way as loading the small molecules. However, in order to compute free energies of binding, we have to make sure that the ligand is aligned with the protein and in an appropriate binding site. BioSimSpace is not a docking program. Therefore ligands will have to be aligned for alchemical free energy calculations in a different way. First, we will load the protein and ligands to check if they are aligned correctly for the calculation. 

In [ ]:
# Load the protein and two ligands.
lysozyme = BSS.IO.readMolecules("input/protein.pdb")[0]
benzene = BSS.IO.readMolecules("input/benzene.mol2")[0]
o_xylene = BSS.IO.readMolecules("input/o-xylene.mol2")[0]

In [ ]:
# Combine the molecules into a single container.
molecules = lysozyme + benzene + o_xylene

In [ ]:
# Create a view to visualise the molecules.
view = BSS.Notebook.View(molecules)
# View the entire system.
view.system()

### 4.2 Parametrisation
<a id="param"></a>

For the protein, we can use a standard Amber forcefield such as `Amber 14 SB`:

In [ ]:
lysozyme = BSS.Parameters.ff14SB(lysozyme).getMolecule()

For the two ligands we can chose to parametrise them using `gaff2`. 

In [ ]:
benzene = BSS.Parameters.gaff2(benzene).getMolecule()
o_xylene = BSS.Parameters.gaff2(o_xylene).getMolecule()

### 5.3 Morphing again

Now all we have to do is go back through the morphing process and then combine the system. 

In [ ]:
mapping = BSS.Align.matchAtoms(o_xylene, benzene)

In [ ]:
# Align o_xylene to benzene based on the mapping.
o_xylene = BSS.Align.rmsdAlign(o_xylene, benzene, mapping)

# Merge the two ligands based on the mapping.
merged = BSS.Align.merge(o_xylene, benzene, mapping)

Next we need to create a composite system containing the merged molecule and the protein:

In [ ]:
system = merged + lysozyme

### 5.4 Binding free energy simulation
Now we can solvate and set up the binding free energy simulation. It looks very similar to the solvation one.

In [ ]:
# Solvate in a 60 angstrom box of TIP3P water.
solvated = BSS.Solvent.tip3p(molecule=system, box=3*[60*BSS.Units.Length.angstrom])

# Create the free energy protocol.
protocol = BSS.Protocol.FreeEnergy(runtime=4*BSS.Units.Time.nanosecond, num_lam=9)

# Initialise the binding free energy object. (Once again, this defaults to the SOMD engine.)
freenrg = BSS.FreeEnergy.Binding(solvated, protocol, work_dir="Binding_o_xylene_benzene" )

`freenrg.run()`
Would again run the simulation again. 

### 5.5 Exercises
<a id="exerc2"></a>
Exercises for binding free energies. 

### 5.5.1 Looking at the directories
Just take a look at the directory that was generated using `BSS.FreeEnergy.Binding()`.
What are the differences and similarities you can observe between the solvation free energies setup and binding free energy setup? 

### 5.5.2 Box sizes of the solvated leg
One thing you can notice is that the box size of the bound and free leg are the same. This of course is a bit silly, because you don't need to use such a large box for just running the ligand in water. There is a handy way of adjusting this. 

In [ ]:
# Solvate in a 60 angstrom box of TIP3P water.
solvated = BSS.Solvent.tip3p(molecule=system, box=3*[60*BSS.Units.Length.angstrom])

# Create the free energy protocol.
protocol = BSS.Protocol.FreeEnergy(runtime=4*BSS.Units.Time.nanosecond, num_lam=9)

# Try using a work_dir called "exercise_5" and a box size for the free leg of the simulations of 30 Angstrom. 
freenrg = BSS.FreeEnergy.Binding(solvated, protocol, work_dir="exercise_5", box=3*[30*BSS.Units.Length.angstrom])

### 5.5.3. Running a minimisation and equilibration before the production
While `SOMD` will automatically minimise and equilibrate the system as part of the free energy simulation, GROMACS will not. You can use BioSimSpace to do this.

**Hint**: use the [documentation](https://biosimspace.org/). Also when returning the system use `getSystem(block=True)` so that we wait for the minimisation and equilibration simulations to finish before returning the system.

In [ ]:
# Solvate in a 60 angstrom box of TIP3P water.
solvated = BSS.Solvent.tip3p(molecule=system, box=3*[60*BSS.Units.Length.angstrom])

# Minimise the system.
minimised = BSS.Process.Gromacs(solvated, BSS.Protocol.Minimisation()).start().getSystem(block=True)

# Equilibrate the system.
equilibrated = BSS.Process.Gromacs(minimised, BSS.Protocol.Equilibration()).start().getSystem(block=True)


# Create the free energy protocol.
protocol = BSS.Protocol.FreeEnergy(runtime=4*BSS.Units.Time.nanosecond, num_lam=9)

# Initialise the binding free energy object.
freenrg = BSS.FreeEnergy.Binding(equilibrated, protocol, work_dir="exercise_5_3", engine="gromacs")